In [26]:
%pip install osmnx networkx geopandas folium shapely numpy ipyleaflet


Note: you may need to restart the kernel to use updated packages.


In [28]:
import osmnx as ox
import networkx as nx
import geopandas as gpd
import folium
from shapely.geometry import LineString
import numpy as np

from ipyleaflet import Map, Marker, basemaps, basemap_to_tiles
from ipywidgets import HTML


In [29]:
import os
os.getcwd()
os.chdir('/home/kuldeep-chamoli/Desktop/GIS/GIS/')

In [30]:
G = ox.load_graphml("data/processed/roads_with_env.graphml")

for u, v, k, d in G.edges(keys=True, data=True):
    d["ndvi"]   = float(d.get("ndvi") or 0.0)
    d["aqi"]    = float(d.get("aqi") or 0.0)
    d["length"] = float(d.get("length") or 1.0)

print("Graph loaded:", len(G.nodes), "nodes |", len(G.edges), "edges")


Graph loaded: 154475 nodes | 392383 edges


In [31]:
def greenness_cost(ndvi, length):
    ndvi = max(0, min(1, ndvi))
    return length * (1 - ndvi)

def pollution_cost(aqi, length):
    return length * (1 + (aqi / 100))

def hybrid_cost(ndvi, aqi, length, w_ndvi=0.7, w_aqi=0.3):
    return w_ndvi * greenness_cost(ndvi, length) + \
           w_aqi * pollution_cost(aqi, length)

def assign_green_cost(G, w_ndvi=0.7, w_aqi=0.3):
    for u, v, k, d in G.edges(keys=True, data=True):
        d["green_cost"] = hybrid_cost(d["ndvi"], d["aqi"], d["length"], w_ndvi, w_aqi)

assign_green_cost(G)
print("Green cost assigned.")


Green cost assigned.


In [7]:
def greenness_cost(ndvi, length):
    ndvi = max(0.0, min(1.0, ndvi))
    return length * (1 - ndvi)

def pollution_cost(aqi, length):
    return length * (1 + (aqi / 100))

def hybrid_cost(ndvi, aqi, length, w_ndvi=0.7, w_aqi=0.3):
    return w_ndvi * greenness_cost(ndvi, length) + \
           w_aqi * pollution_cost(aqi, length)

def assign_edge_costs(G, w_ndvi=0.7, w_aqi=0.3):
    for _, _, _, d in G.edges(keys=True, data=True):
        d["green_cost"] = hybrid_cost(d["ndvi"], d["aqi"], d["length"], w_ndvi, w_aqi)

assign_edge_costs(G)
print("Green cost assigned.")


Green cost assigned.


In [32]:
boundary = gpd.read_file("data/raw/bengaluru_boundary.geojson")
center = boundary.geometry.centroid.iloc[0]
center_lat, center_lon = center.y, center.x

(center_lat, center_lon)


/tmp/ipykernel_33422/348072979.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = boundary.geometry.centroid.iloc[0]


(12.976302529390086, 77.60301428375439)

In [33]:
origin = None
destination = None
origin_marker = None
destination_marker = None
click_count = 0

mclick = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=(center_lat, center_lon),
    zoom=12
)

msg = HTML("<b>Click on the map to select ORIGIN</b>")
mclick.add(msg)

def handle_click(**kwargs):
    global click_count, origin, destination, origin_marker, destination_marker

    if kwargs.get("type") == "click":
        lat = kwargs.get("lat")
        lon = kwargs.get("lng")
        click_count += 1
        
        # ORIGIN
        if click_count == 1:
            origin = (lat, lon)
            origin_marker = Marker(location=origin, draggable=False)
            mclick.add(origin_marker)
            msg.value = f"<b>Origin set:</b> {origin}<br>Now click for DESTINATION."
        
        # DESTINATION
        elif click_count == 2:
            destination = (lat, lon)
            destination_marker = Marker(location=destination, draggable=False)
            mclick.add(destination_marker)
            msg.value = (
                f"<b>Destination set:</b> {destination}<br>"
                "<b>Both points selected!</b>"
            )
            
            mclick.on_interaction(lambda **kw: None)  # disable further clicks

mclick.on_interaction(handle_click)

mclick


Map(center=[12.976302529390086, 77.60301428375439], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [24]:
import geopandas as gpd

# Load Bengaluru boundary
boundary = gpd.read_file("data/raw/bengaluru_boundary.geojson")

# Compute approximate center of the city
city_center = boundary.geometry.centroid.iloc[0]
center_lat, center_lon = city_center.y, city_center.x

center_lat, center_lon


/tmp/ipykernel_33422/389038564.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  city_center = boundary.geometry.centroid.iloc[0]


(12.976302529390086, 77.60301428375439)

In [40]:
print("Enter coordinates in the format: lat, lon")

# Origin input
orig_input = input("Enter ORIGIN (lat,lon): ").strip()
origin = tuple(map(float, orig_input.split(",")))

# Destination input
dest_input = input("Enter DESTINATION (lat,lon): ").strip()
destination = tuple(map(float, dest_input.split(",")))

print("\nSelected Points:")
print("Origin     :", origin)
print("Destination:", destination)


Enter coordinates in the format: lat, lon

Selected Points:
Origin     : (12.9352, 77.6245)
Destination: (12.9784, 77.6408)


In [41]:
origin, destination


((12.9352, 77.6245), (12.9784, 77.6408))

In [43]:
def nearest_node(G, lat, lon):
    return ox.distance.nearest_nodes(G, lon, lat)

def compute_path(G, origin, destination, weight):
    o = nearest_node(G, *origin)
    d = nearest_node(G, *destination)
    return nx.shortest_path(G, o, d, weight=weight)


In [44]:
shortest = compute_path(G, origin, destination, "length")
greenest = compute_path(G, origin, destination, "green_cost")

print("Shortest path nodes:", len(shortest))
print("Greenest path nodes:", len(greenest))


Shortest path nodes: 41
Greenest path nodes: 41


In [46]:
def path_to_linestring(G, path):
    coords = []
    for u, v in zip(path[:-1], path[1:]):
        data = G.get_edge_data(u, v)
        k = list(data.keys())[0]
        geom = data[k].get("geometry")
        if geom:
            coords.extend(list(geom.coords))
        else:
            coords.append((G.nodes[u]["x"], G.nodes[u]["y"]))
    return LineString(coords)

ls_short = path_to_linestring(G, shortest)
ls_green = path_to_linestring(G, greenest)


In [47]:
m = folium.Map(location=origin, zoom_start=13)

folium.GeoJson(
    ls_short,
    name="Shortest Route",
    style_function=lambda x: {"color": "red", "weight": 5}
).add_to(m)

folium.GeoJson(
    ls_green,
    name="Greenest Route",
    style_function=lambda x: {"color": "green", "weight": 5}
).add_to(m)

folium.LayerControl().add_to(m)
m


In [48]:
def metrics(G, path):
    lengths, ndvis, aqis = [], [], []
    for u, v in zip(path[:-1], path[1:]):
        d = G.get_edge_data(u, v)[0]
        lengths.append(d["length"])
        ndvis.append(d["ndvi"])
        aqis.append(d["aqi"])
    return {
        "distance_m": sum(lengths),
        "avg_ndvi": np.mean(ndvis),
        "avg_aqi": np.mean(aqis),
    }

print("Shortest Route Metrics:", metrics(G, shortest))
print("Greenest Route Metrics:", metrics(G, greenest))


Shortest Route Metrics: {'distance_m': 6010.956704310985, 'avg_ndvi': np.float64(0.12729733532760293), 'avg_aqi': np.float64(13.733944954128441)}
Greenest Route Metrics: {'distance_m': 6010.956704310985, 'avg_ndvi': np.float64(0.12729733532760293), 'avg_aqi': np.float64(13.733944954128441)}
